# Module 1 - Create a Kubernetes Cluster

## Objectives

- Learn what a Kubernetes cluster is.
- Learn what Kubectl is
- Discover your learning environment

## Kubernetes Clusters

**Kubernetes coordinates a highly available cluster of computers that are connected to work as a single unit.** The abstractions in Kubernetes allow you to deploy containerized applications to a cluster without tying them specifically to individual machines. To make use of this new model of deployment, applications need to be packaged in a way that decouples them from individual hosts: they need to be containerized. Containerized applications are more flexible and available than in past deployment models, where applications were installed directly onto specific machines as packages deeply integrated into the host. **Kubernetes automates the distribution and scheduling of application containers across a cluster in a more efficient way.** Kubernetes is an open-source platform and is production-ready.

A Kubernetes cluster consists of two types of resources:

- The **Master** coordinates the cluster
- **Nodes** are the workers that run applications

>Kubernetes is a production-grade, open-source platform that orchestrates the placement (scheduling) and execution of application containers within and across computer clusters.

## Cluster Diagram

<img src="https://d33wubrfki0l68.cloudfront.net/99d9808dcbf2880a996ed50d308a186b5900cec9/40b94/docs/tutorials/kubernetes-basics/public/images/module_01_cluster.svg"  width="600" title="test" alt="">

**The Master is responsible for managing the cluster.** The master coordinates all activities in your cluster, such as scheduling applications, maintaining applications' desired state, scaling applications, and rolling out new updates.

**A node is a VM or a physical computer that serves as a worker machine in a Kubernetes cluster.** Each node has a Kubelet, which is an agent for managing the node and communicating with the Kubernetes master. The node should also have tools for handling container operations, such as containerd or Docker. A Kubernetes cluster that handles production traffic should have a minimum of three nodes.

>Masters manage the cluster and the nodes that are used to host the running applications.

When you deploy applications on Kubernetes, you tell the master to start the application containers. The master schedules the containers to run on the cluster's nodes. **The nodes communicate with the master using the [Kubernetes API](https://kubernetes.io/docs/concepts/overview/kubernetes-api/),** which the master exposes. End users can also use the Kubernetes API directly to interact with the cluster.

A Kubernetes cluster can be deployed on either physical or virtual machines. To get started with Kubernetes development, we will use HPE Ezmeral Container Platform. For this tutorial, however, HPE Ezmeral Container Platform is pre-installed.

Now that you know what Kubernetes is, let's go to the online tutorial and start our first cluster!

### Lab 1 - Create a Kubernetes Cluster
#### Step 1 of 3

- Cluster up and running

We already installed HPE Ezmeral Container Platform for you. First login to the platfor to collect context details.

In [1]:
# Login
#
# Environment variables for your working tenant context
#
studentId="student1086" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student1086" # your HPE ECP tenant login credentials - the username is your jupyterHub account username
password="password" # your HPE ECP tenant login credentials - the password is your jupyterHub account password

gateway_host="gateway2.hpedevlab.net"
controller_endpoint="${gateway_host}:8080"


In [2]:

 
tenantname="K8sHackTenant" #case sensitive
 
echo "your operation context is:" $username "on tenant" $tenantname
echo "your REST API endpoint is:" $controller_endpoint

sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId


your operation context is: student1086 on tenant K8sHackTenant
your REST API endpoint is: gateway2.hpedevlab.net:8080
This is your session location:  /api/v2/session/fe650a65-a09b-482a-9c54-cb4a2d6eafd7
This is your session_Id: fe650a65-a09b-482a-9c54-cb4a2d6eafd7


#### Step 2 of 3

To interact with Kubernetes during this bootcamp we’ll use the command line interface, kubectl. We’ll explain kubectl in detail in the next modules, but for now, we’re just going to download a copy of kubectl tailored for your context. To check if kubectl is installed you can run the kubectl version command, then we can check what is our context.

In [3]:
# Get kubectl
mkdir -p ~/.kube
ls ~/.kube
rm -f ~/.kube/config
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ~/.kube/config

echo "Kubeconfig file fetched"
kubectl version
echo "Current context"
kubectl config current-context

Kubeconfig file fetched
Client Version: version.Info{Major:"1", Minor:"19", GitVersion:"v1.19.2", GitCommit:"f5743093fd1c663cb0cbc89748f730662345d44d", GitTreeState:"clean", BuildDate:"2020-09-16T13:41:02Z", GoVersion:"go1.15", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"17", GitVersion:"v1.17.9", GitCommit:"4fb7ed12476d57b8437ada90b4f93b17ffaeed99", GitTreeState:"clean", BuildDate:"2020-07-15T16:10:45Z", GoVersion:"go1.13.9", Compiler:"gc", Platform:"linux/amd64"}
Current context
k8scluster1-K8sHackTenant-student1086


#### Step 3 of 3

- Cluster details

Let’s view the cluster details. We’ll do that by running kubectl cluster-info:

In [4]:
kubectl cluster-info

Kubernetes master is running at https://gateway2.etc.fr.comm.hpecorp.net:9500
KubeDNS is running at https://gateway2.etc.fr.comm.hpecorp.net:9500/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://gateway2.etc.fr.comm.hpecorp.net:9500/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


During this tutorial, we’ll be focusing on the command line for deploying and exploring our application. To view the nodes in the cluster, run the kubectl get nodes command:

In [5]:
kubectl get nodes

NAME                               STATUS   ROLES    AGE   VERSION
worker26.etc.fr.comm.hpecorp.net   Ready    master   97d   v1.17.9
worker27.etc.fr.comm.hpecorp.net   Ready    worker   47d   v1.17.9
worker28.etc.fr.comm.hpecorp.net   Ready    worker   97d   v1.17.9


This command shows all nodes that can be used to host our applications. We have a 3-node cluster, and we can see that its status is ready (it is ready to accept applications for deployment).

In [6]:
kubectl config view

apiVersion: v1
clusters:
- cluster:
    insecure-skip-tls-verify: true
    server: https://gateway2.etc.fr.comm.hpecorp.net:9500
  name: k8scluster1
- cluster:
    insecure-skip-tls-verify: true
    server: https://gateway2.etc.fr.comm.hpecorp.net:9501
  name: k8scluster2
contexts:
- context:
    cluster: k8scluster1
    namespace: k8shacktenant
    user: HPECP-student1086
  name: k8scluster1-K8sHackTenant-student1086
- context:
    cluster: k8scluster1
    namespace: splunk
    user: HPECP-student1086
  name: k8scluster1-splunk-student1086
- context:
    cluster: k8scluster1
    user: HPECP-student1086
  name: k8scluster1-student1086
- context:
    cluster: k8scluster2
    namespace: k8smltenant
    user: HPECP-student1086
  name: k8scluster2-K8sMLTenant-student1086
- context:
    cluster: k8scluster2
    user: HPECP-student1086
  name: k8scluster2-student1086
current-context: k8scluster1-K8sHackTenant-student1086
kind: Config
preferences: {}
users:
- name: HPECP-student1086
  user:
 

This terminates module 1. Let's continue with [module 2, Deploy an app](./3-WKSHP-K8S101-mod2.ipynb).